In [35]:
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
service = Service(executable_path="C:\Program Files (x86)\msedgedriver.exe")
driver = webdriver.Edge(service=service)

# URL of the webpage containing the dropdown menu
url = "https://sportsdata.usatoday.com/hockey/nhl/standings?season=2025&type=conference"
driver.get(url)
time.sleep(5)

<>:10: SyntaxWarning: invalid escape sequence '\P'
<>:10: SyntaxWarning: invalid escape sequence '\P'
C:\Users\admin\AppData\Local\Temp\ipykernel_18272\3059105383.py:10: SyntaxWarning: invalid escape sequence '\P'
  service = Service(executable_path="C:\Program Files (x86)\msedgedriver.exe")


In [36]:
# REGULAR SEASON PREDICTIONS #
# Get path to dropdown
drop_down = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div[4]/div/div[1]/div[1]/div[2]/div/button')

# Scroll to have dropdown in view
driver.execute_script("arguments[0].scrollIntoView();", drop_down)

# Click button
drop_down.click()

In [ ]:
# Wait for year list to load
options_box = WebDriverWait(driver, 2).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[3]/div[4]/div/div[1]/div[1]/div[2]/div/ul'))
)

# Collect past data starting from 2024
years = [year.text for year in options_box.find_elements(By.TAG_NAME, 'li')][1:21]

# Get data for each year
table_data = []
for year in years:
    # Visit the site for every year
    url = f"https://sportsdata.usatoday.com/hockey/nhl/standings?season={year}&type=conference"
    driver.get(url)

    
    # Find the table on the page
    table = WebDriverWait(driver, 2).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[3]/div[4]/div/div[1]/div[2]/div/table'))
    )

    # Store the table data in a variable for looping
    tbody = table.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div[4]/div/div[1]/div[2]/div/table/tbody')
    rows = WebDriverWait(driver, 2).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
    )
    
    print(f"Adding {year} data!")
    for row in rows:
        try:
            row_cells = row.find_elements(By.TAG_NAME, 'td')
            # Check if the row contains cells (data) before processing
            if row_cells:
                row_text = [cell.text for cell in row_cells]
                row_text.append(year)
                table_data.append(row_text) 
                # print(f"Adding {year} data for {row_text[0]}")
        except:
            # Handle StaleElementReferenceException by refinding the row
            rows = WebDriverWait(driver, 2).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            )

# Get table headers
headers = table.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[4]/div/div[1]/div[2]/div/table/thead').text.split(' ')
headers.insert(0,'RECORD')
headers.insert(0,'TEAM')
headers.append('SEASON')

# Close the browser window
driver.quit()  

Adding 2024 data!
Adding 2023 data!
Adding 2022 data!
Adding 2021 data!
Adding 2020 data!
Adding 2019 data!
Adding 2018 data!
Adding 2017 data!
Adding 2016 data!
Adding 2015 data!
Adding 2014 data!
Adding 2013 data!
Adding 2012 data!
Adding 2011 data!
Adding 2010 data!
Adding 2009 data!
Adding 2008 data!
Adding 2007 data!
Adding 2006 data!
Adding 2005 data!


In [38]:
import pandas as pd

# Getting the headers from the data array
# It is important to remove them from the data array afterwards in order to be parsed correctly by Pandas
team_stats = pd.DataFrame(table_data, columns=headers)
team_stats

,TEAM,RECORD,GP,W,L,OTL,PTS,ROW,SOW,SOL,GF,GA,DIFF,HOME,AWAY,L10,STRK,SEASON
0,Washington,C,82,51,22,9,111,49,2,3,288,232,+56,26-9-6,25-13-3,4-6-0,L1,2024
1,Toronto,D,82,52,26,4,108,51,1,2,268,231,+37,27-13-1,25-13-3,9-1-0,W5,2024
2,Tampa Bay,P,82,47,27,8,102,45,2,3,294,219,+75,29-8-4,18-19-4,5-2-3,L1,2024
3,Carolina,P,82,47,30,5,99,47,0,3,266,233,+33,31-9-1,16-21-4,3-6-1,L3,2024
4,Florida,P,82,47,31,4,98,41,6,0,252,223,+29,27-12-2,20-19-2,3-6-1,L2,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,Minnesota,,82,38,36,8,84,33,5,3,231,215,+16,23-16-2,15-20-6,5-4-1,L2,2005
608,Phoenix,,82,38,39,5,81,34,4,3,246,271,-25,19-18-4,19-21-1,4-5-1,W1,2005
609,Columbus,,82,35,43,4,74,27,8,3,223,279,-56,23-18-0,12-25-4,6-3-1,W1,2005
610,Chicago,,82,26,43,13,65,24,2,6,211,285,-74,16-19-6,10-24-7,3-5-2,W2,2005


# Cleaning

In [39]:
# Check data types
team_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TEAM    612 non-null    object
 1   RECORD  612 non-null    object
 2   GP      612 non-null    object
 3   W       612 non-null    object
 4   L       612 non-null    object
 5   OTL     612 non-null    object
 6   PTS     612 non-null    object
 7   ROW     612 non-null    object
 8   SOW     612 non-null    object
 9   SOL     612 non-null    object
 10  GF      612 non-null    object
 11  GA      612 non-null    object
 12  DIFF    612 non-null    object
 13  HOME    612 non-null    object
 14  AWAY    612 non-null    object
 15  L10     612 non-null    object
 16  STRK    612 non-null    object
 17  SEASON  612 non-null    object
dtypes: object(18)
memory usage: 86.2+ KB


In [40]:
team_stats.columns

Index(['TEAM', 'RECORD', 'GP', 'W', 'L', 'OTL', 'PTS', 'ROW', 'SOW', 'SOL',
       'GF', 'GA', 'DIFF', 'HOME', 'AWAY', 'L10', 'STRK', 'SEASON'],
      dtype='object')

In [41]:
# Remove '+' from DIFF column
team_stats['DIFF'] = team_stats['DIFF'].str.replace('+', '')

# Convert following columns to floats
string_columns = ['GP', 'W', 'L', 'OTL', 'PTS', 'ROW', 'SOW', 'SOL', 'GF', 'GA', 'DIFF']
team_stats[string_columns] = team_stats[string_columns].apply(pd.to_numeric, errors='coerce')

team_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TEAM    612 non-null    object
 1   RECORD  612 non-null    object
 2   GP      612 non-null    int64 
 3   W       612 non-null    int64 
 4   L       612 non-null    int64 
 5   OTL     612 non-null    int64 
 6   PTS     612 non-null    int64 
 7   ROW     612 non-null    int64 
 8   SOW     612 non-null    int64 
 9   SOL     612 non-null    int64 
 10  GF      612 non-null    int64 
 11  GA      612 non-null    int64 
 12  DIFF    612 non-null    int64 
 13  HOME    612 non-null    object
 14  AWAY    612 non-null    object
 15  L10     612 non-null    object
 16  STRK    612 non-null    object
 17  SEASON  612 non-null    object
dtypes: int64(11), object(7)
memory usage: 86.2+ KB


In [42]:
file_path = 'nhl_regular_season_team_stats.csv'
team_stats.to_csv(file_path, index=False)